In [2]:
from datasets import load_dataset
import datasets
ds = load_dataset("Anthropic/hh-rlhf")


harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

helpful-rejection-sampled/train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

harmless-base/test.jsonl.gz:   0%|          | 0.00/743k [00:00<?, ?B/s]

helpful-base/test.jsonl.gz:   0%|          | 0.00/875k [00:00<?, ?B/s]

helpful-online/test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

helpful-rejection-sampled/test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

In [7]:
train: datasets.Dataset = ds["train"]
test: datasets.Dataset = ds["test"]
print("total train examples:", len(train))
print("total test examples:", len(test))

total train examples: 160800
total test examples: 8552


In [8]:
import regex as re
def filter(example):
    # filter multi-turn conversations
    chosen = example["chosen"]
    # we want exactly one "Human: " and one "Assistant: ", all need at the start of line
    human_count = len(re.findall(r"^Human: ", chosen, flags=re.MULTILINE))
    assistant_count = len(re.findall(r"^Assistant: ", chosen, flags=re.MULTILINE))
    return human_count == 1 and assistant_count == 1
train = train.filter(filter)
test = test.filter(filter)
print("filtered train examples:", len(train))
print("filtered test examples:", len(test))

Filter:   0%|          | 0/160800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8552 [00:00<?, ? examples/s]

filtered train examples: 49386
filtered test examples: 2609


In [ ]:
def extract_prompt_and_response(example):
    chosen = example["chosen"]
    prompt_match = re.search(r"^Human: (.+?)^Assistant: ", chosen, flags=re.MULTILINE | re.DOTALL)
    response_match = re.search(r"^Assistant: (.+)$", chosen, flags=re.MULTILINE | re.DOTALL)
    prompt = prompt_match.group(1).strip() if prompt_match else ""
    resp_good = response_match.group(1).strip() if response_match else ""
    reject = example["rejected"]
    return_prompt_match = re.search(r"^Human: (.+?)^Assistant: ", reject, flags=re.MULTILINE | re.DOTALL)
    if return_prompt_match:
        return_prompt = return_prompt_match.group(1).strip()
        assert return_prompt == prompt, "Prompt in chosen and rejected do not match!"
    response_match = re.search(r"^Assistant: (.+)$", reject, flags=re.MULTILINE | re.DOTALL)
    resp_bad = response_match.group(1).strip() if response_match else "" 
    return {"prompt": prompt, "good": resp_good, "bad": resp_bad}